# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902034


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/37 [00:00<?, ?it/s]

Rendering models:   5%|▌         | 2/37 [00:07<02:03,  3.53s/it]

Rendering models:   8%|▊         | 3/37 [00:07<01:26,  2.53s/it]

Rendering models:  11%|█         | 4/37 [00:07<01:00,  1.84s/it]

Rendering models:  14%|█▎        | 5/37 [00:07<00:43,  1.35s/it]

...

Rendering models:  89%|████████▉ | 33/37 [00:14<00:01,  3.26it/s]

Rendering models:  92%|█████████▏| 34/37 [00:14<00:00,  3.53it/s]

Rendering models:  95%|█████████▍| 35/37 [00:15<00:00,  3.70it/s]

Rendering models: 100%|██████████| 37/37 [00:15<00:00,  3.90it/s]

equidad1                              0.069229
nickoftona                            0.000160
Thedunkmasta                          0.002372
Thedunkmasta                          0.003161
Thedunkmasta                          0.003642
Thedunkmasta                          0.004122
Thedunkmasta                          0.003161
Thedunkmasta                          0.003661
Thedunkmasta                          0.002604
Thedunkmasta                          0.003166
mlang2017                             0.000313
not-logged-in-769e51a01946aca9871e    0.089563
a.ham.246                             0.000488
not-logged-in-f42e1cbbe5acea82cc50    0.000084
not-logged-in-ea8ed61a7274cfb6a43b    0.028693
not-logged-in-ea8ed61a7274cfb6a43b    0.000715
macaroniman                           0.002126
JPenn550                              0.005019
Linda_J._Berkel                       0.000118
Paddy_van_CH                          0.000255
not-logged-in-a0164d241f30411408f0    0.000282
Mr_DMan      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())